### Resources for finetuning:
https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb
 
https://github.com/huggingface/notebooks/blob/main/examples/question_answering.ipynb
https://github.com/huggingface/transformers/tree/main/notebooks

Paper:
https://arxiv.org/pdf/2205.01068.pdf

In [1]:
from datasets import load_dataset

dataset = load_dataset("pile-of-law/pile-of-law",'r_legaladvice')

Reusing dataset pile_of_law (/Users/thomasrochefort/.cache/huggingface/datasets/pile-of-law___pile_of_law/r_legaladvice/0.0.0/acacf3e29a952ba9026148b979cb438151ebd33f842f5779a213967033c88619)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(dataset["train"])


,text,created_timestamp,downloaded_timestamp,url
0,"Title: I was told I can't come back to school due to medical condition [New York]\nQuestion:[deleted]\nAnswer #1: I want to add a few points: \n\nGrad school is different than undergraduate work. Fields move quickly, in some cases, and, regardless of outside issues, if you haven't reached the point of taking your exams by your fourth year the school may opt not to allow you to return. Keep in mind that being unable to complete coursework does not speak to your ability to complete dissertation work successfully.\n\nAssuming that you are funded, you are likely tying up a funding line for each year you are there, including when you take leave, and the school may not want to continue to fund you. \n\nAs to the crux of your complaint, schools have to make a reasonable accommodation for a medical issue or illness. Letting you take three significant gaps in a four year program is a lot to ask, and may indicate to them that you have attempted to return without being sufficiently prepared. The fact that you considered dropping out of the program may also affect how they see you as a student.Answer #2: INAL but was in a nearly identical situation to you (also had to leave school for diagnosed depression and anxiety disorders). From what I've seen, most schools have a limit on how long they let you leave for medical reasons. At the first school I went to, students were allowed two quarters of personal leave and eight quarters of medical leave. If you had to take time off beyond that, you were (and I was) dropped from the school. Depending on the circumstances, some schools will let you reapply to enter them again, but that's a very school and situation specific thing. \n\nI ended up transferring out of the first school to a community college while I dealt with the medical issue, and then into another four year university to finish my degree. If it comes down to it, you might have to do the same. Answer #3: Part of it could also be their accreditation agency stipulates that you must finish a degree program within a certain length of time. I know at the school I went to I had 1.5 times the length of the program to finish otherwise I would have to start over. It could simply be that you are getting dangerously close to that point and it has nothing to do with the medical condition.Answer #4: Public or private? Even with ADA protection, a private college can essentially drop you for no reasons. That does not mean you're not entitled to pursue legal ramifications, however it tends to be more difficult. \n\nDo you have any scholarships or grants? If so, what? The cost occurred via your leaves may have left a financial burden on the institution.\n\nDid you leaves cause any monetary repercussions? \nSee above.\n\nDid you withdraw past the deadlines for add/drop? You illness may not require said documentation prior to the deadlines, but it may be applicable if suit follows.\n\nEssentially, the college's defense would be along he lines of whether or not they reasonably accommodated your disability, and their arguments would be accompanied by wages lost whilst doing so.",02-23-2016,11-10-2021,https://www.reddit.com/r/legaladvice/comments/476b71/i_was_told_i_cant_come_back_to_school_due_to/
1,"Title: If you're furloughed and you dont want to go back to work, can you continue taking unemployment?\nQuestion:My wife works in retail in Houston, and the store she works at is looking like it's opening up this Friday, they average over 1200 customer interactions a day on an avg weekend. She isn't comfortable going back to that with all the caronavirus stuff still going on, if she didnt return, could she continue to take unemployment until she finds a new career opportunity? Or would it be stopped if they said for her to come back?\nAnswer #1: If they give her her job back, and she chooses not to go, now she's quit and no longer eligible for unemployment.Answer #2: If she's offered her job back and denies 

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m")

Downloading:   0%|          | 0.00/651 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/opt-125m were not used when initializing OPTForCausalLM: ['model.decoder.final_layer_norm.weight', 'model.decoder.final_layer_norm.bias']
- This IS expected if you are initializing OPTForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing OPTForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [5]:
dataset["train"][161]

{'text': "Title: Personal information and walmart\nQuestion:This is whats going on. I bought a nerf gun for my son at Wal-Mart for Christmas. I found the same nerf gun $40 cheaper, so naturally returned it to Wal-Mart. I got a call yesterday (well voicemail) from a Wal-Mart customer who bought the nerf gun.. it gets weird here.. they said Wal-Mart put my personal information on the box and decided to use it to contact me to see if there is anything wrong with it. I am naturally paranoid with my personal information as is most people. Why would they do this? What should I do? sorry if I am in the wrong sub-reddit.. Thanks in advance \n\np.s. I do not frequent Wal-Mart often, it was the only place for this particular nerf gun at the moment \n\nI am in the chicago area \nAnswer #1: Your information is personal in nature, but is by no means private.  Your full name, birth date, home address, and sometimes phone numbers are all generally a matter of public record and readily available to an

In [6]:
tokenize_function(dataset["train"][161])

{'input_ids': [2, 46525, 35, 13129, 335, 8, 21788, 15964, 50118, 45641, 35, 713, 16, 45676, 164, 15, 4, 38, 2162, 10, 38286, 506, 1751, 13, 127, 979, 23, 6092, 12, 13012, 13, 1619, 4, 38, 303, 5, 276, 38286, 506, 1751, 68, 1749, 7246, 6, 98, 8366, 1835, 24, 7, 6092, 12, 13012, 4, 38, 300, 10, 486, 2350, 36, 3056, 30118, 10555, 43, 31, 10, 6092, 12, 13012, 2111, 54, 2162, 5, 38286, 506, 1751, 7586, 24, 1516, 7735, 259, 7586, 51, 26, 6092, 12, 13012, 342, 127, 1081, 335, 15, 5, 2233, 8, 1276, 7, 304, 24, 7, 1511, 162, 7, 192, 114, 89, 16, 932, 1593, 19, 24, 4, 38, 524, 8366, 33554, 19, 127, 1081, 335, 25, 16, 144, 82, 4, 2612, 74, 51, 109, 42, 116, 653, 197, 38, 109, 116, 6661, 114, 38, 524, 11, 5, 1593, 2849, 12, 48724, 7586, 4557, 11, 3316, 1437, 50118, 50118, 642, 4, 29, 4, 38, 109, 45, 7690, 6092, 12, 13012, 747, 6, 24, 21, 5, 129, 317, 13, 42, 1989, 38286, 506, 1751, 23, 5, 1151, 1437, 50118, 50118, 100, 524, 11, 5, 1855, 42938, 443, 1437, 50118, 33683, 849, 134, 35, 2486, 335, 16, 

In [7]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=8, remove_columns=["text","created_timestamp","downloaded_timestamp","url"])

#0:   0%|          | 0/14 [00:00<?, ?ba/s]

#3:   0%|          | 0/14 [00:00<?, ?ba/s]

#4:   0%|          | 0/14 [00:00<?, ?ba/s]

#5:   0%|          | 0/14 [00:00<?, ?ba/s]

#1:   0%|          | 0/14 [00:00<?, ?ba/s]

...

#2:   0%|          | 0/5 [00:00<?, ?ba/s]

#6:   0%|          | 0/5 [00:00<?, ?ba/s]

#3:   0%|          | 0/5 [00:00<?, ?ba/s]

#4:   0%|          | 0/5 [00:00<?, ?ba/s]

#7:   0%|          | 0/5 [00:00<?, ?ba/s]

In [8]:
tokenized_dataset["train"][1]

{'input_ids': [2,
  46525,
  35,
  38,
  206,
  951,
  16,
  3433,
  88,
  127,
  512,
  7,
  120,
  13543,
  13,
  162,
  145,
  10,
  1099,
  1393,
  411,
  377,
  536,
  50118,
  45641,
  35,
  713,
  16,
  11,
  2042,
  4,
  3727,
  11,
  392,
  6,
  38,
  13636,
  4024,
  5,
  1593,
  169,
  159,
  10,
  5342,
  3610,
  65,
  169,
  921,
  142,
  38,
  56,
  45,
  551,
  127,
  33286,
  5679,
  29,
  14,
  183,
  4,
  17397,
  117,
  1677,
  58,
  567,
  1567,
  162,
  8,
  25,
  1010,
  25,
  38,
  5426,
  24,
  6,
  38,
  4024,
  11,
  5,
  4806,
  6625,
  13,
  5,
  1079,
  9,
  5,
  1803,
  454,
  38,
  115,
  1004,
  4,
  38,
  21,
  2198,
  7619,
  8435,
  66,
  59,
  24,
  6,
  98,
  38,
  1276,
  7,
  213,
  124,
  184,
  8,
  38003,
  2185,
  4,
  38,
  1299,
  3668,
  6587,
  59,
  24,
  4,
  287,
  38,
  21,
  1428,
  184,
  6,
  38,
  5324,
  10,
  512,
  511,
  162,
  4,
  38,
  554,
  7,
  185,
  124,
  3197,
  8,
  1076,
  21242,
  7,
  860,
  7,
  2217,
  123,
  13

In [9]:
#block_size = tokenizer.model_max_length
block_size = 128

In [10]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [11]:
lm_datasets = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=8,
)

#0:   0%|          | 0/14 [00:00<?, ?ba/s]

#1:   0%|          | 0/14 [00:00<?, ?ba/s]

#5:   0%|          | 0/14 [00:00<?, ?ba/s]

#3:   0%|          | 0/14 [00:00<?, ?ba/s]

#4:   0%|          | 0/14 [00:00<?, ?ba/s]

...

#7:   0%|          | 0/5 [00:00<?, ?ba/s]

#6:   0%|          | 0/5 [00:00<?, ?ba/s]

#3:   0%|          | 0/5 [00:00<?, ?ba/s]

#5:   0%|          | 0/5 [00:00<?, ?ba/s]

#4:   0%|          | 0/5 [00:00<?, ?ba/s]

In [12]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
        f"opt-test",

    no_cuda= True,
    bf16=True,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,

    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

Using cpu_amp half precision backend


In [18]:
trainer.train()


***** Running training *****
  Num examples = 405492
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 152061


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [14]:
import torch

In [17]:
torch.get_num_threads()

8